In [1]:
!pip install dash

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/10.2 MB 68.3 MB/s eta 0:00:00


In [4]:
!wget "https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_dash.csv"

--2023-12-08 10:00:40--  https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_dash.csv
Resolving cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud (cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud)... 198.23.119.245
Connecting to cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud (cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud)|198.23.119.245|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2476 (2.4K) [text/csv]
Saving to: ‘spacex_launch_dash.csv’

spacex_launch_dash. 100%[===================>]   2.42K  --.-KB/s    in 0s      

2023-12-08 10:00:40 (1.71 GB/s) - ‘spacex_launch_dash.csv’ saved [2476/2476]



In [7]:
import numpy as np
import pandas as pd
import dash
from dash import html, dcc
import dash_html_components as html
import dash_core_components as dcc
from dash.dependencies import Input, Output
import plotly.express as px

In [31]:
spacex_df=pd.read_csv('spacex_launch_dash.csv')
max_payload = spacex_df['Payload Mass (kg)'].max()
min_payload = spacex_df['Payload Mass (kg)'].min()

spacex_df.tail(20)


,Unnamed: 0,Flight Number,Launch Site,class,Payload Mass (kg),Booster Version,Booster Version Category
36,36,30,KSC LC-39A,1,2490.00,F9 FT B1031.1,FT
37,37,31,KSC LC-39A,0,5600.00,F9 FT B1030,FT
38,38,32,KSC LC-39A,1,5300.00,F9 FT B1021.2,FT
39,39,33,KSC LC-39A,1,3696.65,F9 FT B1032.1,FT
40,40,34,KSC LC-39A,0,6070.00,F9 FT B1034,FT
41,41,35,KSC LC-39A,1,2708.00,F9 FT B1035.1,FT
42,42,36,KSC LC-39A,1,3669.00,F9 FT B1029.2,FT
43,43,38,KSC LC-39A,0,6761.00,F9 FT B1037,FT
44,44,39,KSC LC-39A,1,3310.00,F9 B4 B1039.1,B4
45,45,41,KSC LC-39A,1,4990.00,F9 B4 B1040.1,B4


In [73]:
pie_data=spacex_df.groupby('Launch Site')['class'].mean()
pie_data=pie_data.reset_index()
pie_data1=pie_data[pie_data['Launch Site']=='KSC LC-39A'].reset_index()
#pie_data1=pie_data1.reset_index()
pie_data1
a=pie_data1['class'][0]
value=[a,1-a]
label=[1,0]



In [100]:
fil_data=spacex_df[spacex_df['Launch Site']=='CCAFS SLC-40'].reset_index()

plot_data=fil_data[fil_data['Payload Mass (kg)']<10000]
plot_data=plot_data[plot_data['Payload Mass (kg)']>10]

px.scatter(plot_data, x='Payload Mass (kg)', y='class', color='Booster Version Category')

In [106]:
# Create a dash application
app = dash.Dash(__name__)

# Create an app layout
app.layout = html.Div(children=[
    html.H1('SpaceX Launch Records Dashboard',style={'textAlign': 'center', 'color': '#503D36','font-size': 40}),
# TASK 1: Add a dropdown list to enable Launch Site selection
# The default select value is for ALL sites
    html.Div([html.Label("Select a Launch Site"),
        dcc.Dropdown(id='site-dropdown',
              options=[{'label': 'All Sites', 'value': 'ALL'},
               {'label': 'CCAFS LC-40', 'value': 'CCAFS LC-40'},{'label': 'VAFB SLC-4E', 'value': 'VAFB SLC-4E'},
               {'label': 'KSC LC-39A', 'value': 'KSC LC-39A'},{'label': 'CCAFS SLC-40', 'value': 'CCAFS SLC-40'}
                  ],
              value='ALL',
              placeholder='Select a Launch Site here',
              searchable= True
              )]),
    html.Br(),
# TASK 2: Add a pie chart to show the total successful launches count for all sites
# If a specific launch site was selected, show the Success vs. Failed counts for the site
    html.Div(dcc.Graph(id='success-pie-chart')),
    html.Br(),

    html.P("Payload range (Kg):"),
# TASK 3: Add a slider to select payload range
    html.Div([html.Label("Select a Range"),
        dcc.RangeSlider(id='payload-slider',
                min=0, max=10000, step=1000,
                marks={0: '0', 100: '100'},
                value=[min_payload, max_payload],)
        ]),

# TASK 4: Add a scatter chart to show the correlation between payload and launch success
    html.Div(dcc.Graph(id='success-payload-scatter-chart')),
    ])

# TASK 2:
# Add a callback function for `site-dropdown` as input, `success-pie-chart` as output
# Function decorator to specify function input and output
@app.callback(Output(component_id='success-pie-chart', component_property='figure'),
       Input(component_id='site-dropdown', component_property='value'))
def get_pie_chart(entered_site):
    filtered_df = spacex_df
    if entered_site == 'ALL':
        fig = px.pie(filtered_df, values='class', names='Launch Site',
        title='Total Success Launch by Sites')
        return fig
    else:
        # return the outcomes piechart for a selected site
        pie_data=filtered_df.groupby('Launch Site')['class'].mean()
        pie_data=pie_data.reset_index()
        pie_data1=pie_data[pie_data['Launch Site']==entered_site].reset_index()
        a=pie_data1['class'][0]
        value=[a,1-a]
        label=[1,0]
        fig = px.pie(values=value, names=label,
        title='Success Launch for Site{}'.format(entered_site))

        return fig

# TASK 4:
# Add a callback function for `site-dropdown` and `payload-slider` as inputs, `success-payload-scatter-chart` as output
@app.callback(Output(component_id='success-payload-scatter-chart', component_property='figure'),
       [Input(component_id='site-dropdown', component_property='value'),
       Input(component_id="payload-slider", component_property="value")])

def get_scat(entered_site,entered_val):
  min_range=entered_val[0]
  max_range=entered_val[1]
  if entered_site =='ALL':
    plot_data=spacex_df[(spacex_df['Payload Mass (kg)']>min_range) & (spacex_df['Payload Mass (kg)']<max_range)]
    fig = px.scatter(plot_data, x='Payload Mass (kg)', y='class', color='Booster Version Category',
        title='Relation between Payload and Success for All Site')
    return fig
  else:
    #fil_data=spacex_df[spacex_df['Launch Site'] == entered_site ]
    plot_data=spacex_df[(spacex_df['Launch Site'] == entered_site) & (spacex_df['Payload Mass (kg)']>min_range)
              & (spacex_df['Payload Mass (kg)']<max_range)]
    fig= px.scatter(plot_data, x='Payload Mass (kg)', y='class', color='Booster Version Category',
            title='Relation between Payload and Success for Site {}'.format(entered_site))
    return fig

# Run the app
if __name__ == '__main__':
    app.run_server()


<IPython.core.display.Javascript object>